In [131]:
# Import necessary libraries
import pandas as pd
import glob
from pandas_profiling import ProfileReport
import numpy as np
import json
import duckdb
import re

# clean oscar data from https://www.kaggle.com/datasets/unanimad/the-oscar-award

In [132]:

oscars = pd.read_csv('data/the_oscar_award.csv')
oscars.isna().sum()

year_film          0
year_ceremony      0
ceremony           0
category           0
name               3
film             310
winner           120
dtype: int64

In [120]:

oscars = oscars.dropna(subset=[ 'winner', 'film'])
oscars.isna().sum()

year_film        0
year_ceremony    0
ceremony         0
category         0
name             0
film             0
winner           0
dtype: int64

In [121]:
oscars

,year_film,year_ceremony,ceremony,category,name,film,winner
0,1927,1928,1,ACTOR,Richard Barthelmess,The Noose,False
1,1927,1928,1,ACTOR,Emil Jannings,The Last Command,True
2,1927,1928,1,ACTRESS,Louise Dresser,A Ship Comes In,False
3,1927,1928,1,ACTRESS,Janet Gaynor,7th Heaven,True
4,1927,1928,1,ACTRESS,Gloria Swanson,Sadie Thompson,False
...,...,...,...,...,...,...,...
10630,2021,2022,94,WRITING (Original Screenplay),Written by Kenneth Branagh,Belfast,True
10631,2021,2022,94,WRITING (Original Screenplay),Screenplay by Adam McKay; Story by Adam McKay ...,Don't Look Up,False
10632,2021,2022,94,WRITING (Original Screenplay),Written by Zach Baylin,King Richard,False
10633,2021,2022,94,WRITING (Original Screenplay),Written by Paul Thomas Anderson,Licorice Pizza,False


In [122]:
#oscars = oscars.drop_duplicates(subset=["film", "year_film"], keep="first") 


oscars['year + title'] = oscars["year_film"].astype(str) + oscars["film"]
oscars

oscars = oscars.drop_duplicates(subset=['year + title'] ,keep='first')
oscars

,year_film,year_ceremony,ceremony,category,name,film,winner,year + title
0,1927,1928,1,ACTOR,Richard Barthelmess,The Noose,False,1927The Noose
1,1927,1928,1,ACTOR,Emil Jannings,The Last Command,True,1927The Last Command
2,1927,1928,1,ACTRESS,Louise Dresser,A Ship Comes In,False,1927A Ship Comes In
3,1927,1928,1,ACTRESS,Janet Gaynor,7th Heaven,True,19277th Heaven
4,1927,1928,1,ACTRESS,Gloria Swanson,Sadie Thompson,False,1927Sadie Thompson
...,...,...,...,...,...,...,...,...
10613,2021,2022,94,SHORT FILM (Live Action),Martin Strange-Hansen and Kim Magnusson,On My Mind,False,2021On My Mind
10614,2021,2022,94,SHORT FILM (Live Action),K.D. Dávila and Levin Menekse,Please Hold,False,2021Please Hold
10621,2021,2022,94,VISUAL EFFECTS,"Swen Gillberg, Bryan Grill, Nikos Kalaitzidis ...",Free Guy,False,2021Free Guy
10623,2021,2022,94,VISUAL EFFECTS,"Christopher Townsend, Joe Farrell, Sean Noel W...",Shang-Chi and the Legend of the Ten Rings,False,2021Shang-Chi and the Legend of the Ten Rings


In [123]:
by_cat=oscars.groupby(by='category', as_index=False).agg({'film':'count'})
by_cat=by_cat.sort_values(by='film', ascending=False, ignore_index=True)
print(len(by_cat))
display(by_cat.head(10))

98


,category,film
0,DOCUMENTARY (Short Subject),378
1,DOCUMENTARY (Feature),343
2,FOREIGN LANGUAGE FILM,288
3,ACTOR IN A SUPPORTING ROLE,279
4,ACTOR IN A LEADING ROLE,227
5,ACTOR,222
6,SHORT FILM (Live Action),216
7,SHORT FILM (Animated),205
8,SHORT SUBJECT (Cartoon),169
9,ART DIRECTION,165


In [124]:
#Rewrite all categories in lower-case
oscars['category']=[word.lower() for word in oscars['category']]

oscars

C:\Users\sara_\AppData\Local\Temp\ipykernel_12060\1394384046.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oscars['category']=[word.lower() for word in oscars['category']]


,year_film,year_ceremony,ceremony,category,name,film,winner,year + title
0,1927,1928,1,actor,Richard Barthelmess,The Noose,False,1927The Noose
1,1927,1928,1,actor,Emil Jannings,The Last Command,True,1927The Last Command
2,1927,1928,1,actress,Louise Dresser,A Ship Comes In,False,1927A Ship Comes In
3,1927,1928,1,actress,Janet Gaynor,7th Heaven,True,19277th Heaven
4,1927,1928,1,actress,Gloria Swanson,Sadie Thompson,False,1927Sadie Thompson
...,...,...,...,...,...,...,...,...
10613,2021,2022,94,short film (live action),Martin Strange-Hansen and Kim Magnusson,On My Mind,False,2021On My Mind
10614,2021,2022,94,short film (live action),K.D. Dávila and Levin Menekse,Please Hold,False,2021Please Hold
10621,2021,2022,94,visual effects,"Swen Gillberg, Bryan Grill, Nikos Kalaitzidis ...",Free Guy,False,2021Free Guy
10623,2021,2022,94,visual effects,"Christopher Townsend, Joe Farrell, Sean Noel W...",Shang-Chi and the Legend of the Ten Rings,False,2021Shang-Chi and the Legend of the Ten Rings


In [125]:
def my_map(function_to_apply, list_of_inputs):
    """
    Traverses a list, apply some function to each of the elements and collects the output.
    """
    
    new_list = []
    for i in list_of_inputs:
        new_list.append(function_to_apply(i))
        
    return new_list

In [126]:
def convert_word(word):
    """
    Strips a word from all punctuation, whitespace, and digits. Then converts the word into all lower case
    and convert all characters with accents to non-accent letters.
    """
    word = re.sub(r'[^\w\s]','',word).lower()
    word = re.sub(r'[àáâãäå]', 'a', word)
    word = re.sub(r'[èéêë]', 'e', word)
    word = re.sub(r'[ìíîï]', 'i', word)
    word = re.sub(r'[òóôõö]', 'o', word)
    word = re.sub(r'[ùúûü]', 'u', word)
    
    return word

In [127]:
def map_cleaned_strings(dfname, columnname):
    """
    Maps new values in a df-column. From a new created dictionary, with as Key the 'old' string value and 
    as Value the 'new'-string value. The 'new'-string values are extracted with the function convert_word().
    """
    dict_old_new = {}
    for i in dfname[columnname]:
        if type(i) != str:
            continue
        else:
            dict_old_new[i] = convert_word(i)

    dfname[columnname] = dfname[columnname].map(dict_old_new)
    
    return dfname[columnname]

In [128]:
#call function map_cleaned_strings to apply in the dataframe on the columns title, director, cast, country
map_cleaned_strings(oscars, 'name')
map_cleaned_strings(oscars, 'film')
map_cleaned_strings(oscars, 'name')
map_cleaned_strings(oscars, 'film')

oscars

C:\Users\sara_\AppData\Local\Temp\ipykernel_12060\4069122000.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfname[columnname] = dfname[columnname].map(dict_old_new)
C:\Users\sara_\AppData\Local\Temp\ipykernel_12060\4069122000.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfname[columnname] = dfname[columnname].map(dict_old_new)
C:\Users\sara_\AppData\Local\Temp\ipykernel_12060\4069122000.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usin

,year_film,year_ceremony,ceremony,category,name,film,winner,year + title
0,1927,1928,1,actor,richard barthelmess,the noose,False,1927The Noose
1,1927,1928,1,actor,emil jannings,the last command,True,1927The Last Command
2,1927,1928,1,actress,louise dresser,a ship comes in,False,1927A Ship Comes In
3,1927,1928,1,actress,janet gaynor,7th heaven,True,19277th Heaven
4,1927,1928,1,actress,gloria swanson,sadie thompson,False,1927Sadie Thompson
...,...,...,...,...,...,...,...,...
10613,2021,2022,94,short film (live action),martin strangehansen and kim magnusson,on my mind,False,2021On My Mind
10614,2021,2022,94,short film (live action),kd davila and levin menekse,please hold,False,2021Please Hold
10621,2021,2022,94,visual effects,swen gillberg bryan grill nikos kalaitzidis an...,free guy,False,2021Free Guy
10623,2021,2022,94,visual effects,christopher townsend joe farrell sean noel wal...,shangchi and the legend of the ten rings,False,2021Shang-Chi and the Legend of the Ten Rings


In [129]:
# create a connection to a file called 'file.db'
con = duckdb.connect('imdb_reviews.db')

# create a table and load data into it
con.sql("CREATE TABLE IF NOT EXISTS oscars AS SELECT * FROM oscars")


con.close()

In [130]:
con.close()